In [44]:
import pandas as pd
from unidecode import unidecode

In [45]:
df = pd.read_csv("../Data/Processed/crimes_clear.csv")

In [46]:
# Changing the field "fecha" into a datetime type
df['fecha'] = pd.to_datetime(df['fecha'])

# Adding a new column to know the day of the week that was reporte the crime
df['dia'] = df['fecha'].dt.day_name()

# Changing names to spanish since the whole database is in spanish
day_name_mapping = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

df['dia'] = df['dia'].map(day_name_mapping)

# Importing and cleaning the new DF to know the location

In [47]:
position_df = pd.read_excel("../Data/Raw/DIVIPOLA_Municipios.xlsx", skiprows=10, skipfooter=10)

In [48]:
position_df = position_df.drop(['Código ', 'Código .1', 'Unnamed: 4'], axis=1)

In [49]:
position_df.columns

Index(['Nombre', 'Nombre.1', 'LATITUD', 'LONGITUD'], dtype='object')

In [50]:
# Remove the brackets and make text uppercase to match with the new DF
df.municipio = df.municipio.str.replace(r'\(.*\)', '').str.upper()

# Change the column's name to match in both dataframes
position_df.rename(columns={"Nombre.1": "municipio"}, inplace=True)

/tmp/ipykernel_264/150562150.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.municipio = df.municipio.str.replace(r'\(.*\)', '').str.upper()


In [51]:
df

,arma,departamento,municipio,fecha,genero,grupo_etario,codigo_dane,cantidad,delito,dia
0,ARMA BLANCA / CORTOPUNZANTE,AMAZONAS,LETICIA,2023-06-13,MASCULINO,ADULTOS,91001000,1,amenza,Martes
1,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,AMAGÁ,2023-06-26,FEMENINO,ADULTOS,5030000,1,amenza,Lunes
2,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,ANGOSTURA,2023-05-05,MASCULINO,ADULTOS,5038000,1,amenza,Viernes
3,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,BELLO,2023-02-23,FEMENINO,ADULTOS,5088000,1,amenza,Jueves
4,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,CISNEROS,2023-07-25,FEMENINO,ADULTOS,5190000,1,amenza,Martes
...,...,...,...,...,...,...,...,...,...,...
284402,SIN EMPLEO DE ARMAS,VICHADA,PUERTO CARREÑO,2023-08-27,FEMENINO,ADULTOS,99001000,1,violencia intrafamiliar,Domingo
284403,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALÍA,2023-02-07,FEMENINO,ADULTOS,99624000,1,violencia intrafamiliar,Martes
284404,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALÍA,2023-03-17,FEMENINO,ADULTOS,99624000,1,violencia intrafamiliar,Viernes
284405,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALÍA,2023-03-31,FEMENINO,ADOLESCENTES,99624000,1,violencia intrafamiliar,Viernes


In [52]:
# Some places has an space at the end, this code remove the las space
df["municipio"] = df["municipio"].apply(lambda x: x[:-1] if x.endswith(" ") else x)

# Remove spanish accents and ticks
df["municipio"] =  df["municipio"].apply(unidecode)
position_df["municipio"] = position_df["municipio"].apply(unidecode)

In [53]:
# some places doesnt have athe same name, solvind this
places_to_change = {
"DON MATIAS" : "DONMATIAS",
"SAN VICENTE" : "SAN VICENTE FERRER",
"CARTAGENA" : "CARTAGENA DE INDIAS",
"MOMPOS" : "SANTA CRUZ DE MOMPOX",
"SAN ANDRES SOTAVENTO" : "SAN ANDRES DE SOTAVENTO",
"BOGOTA D.C." : "BOGOTA, D.C.",
"CUCUTA" : "SAN JOSE DE CUCUTA",
"CALI" : "SANTIAGO DE CALI",
"SANTAFE DE ANTIOQUIA" : "SANTA FE DE ANTIOQUIA",
"GUICAN" : "GUICAN DE LA SIERRA",
"CERRO SAN ANTONIO" : "CERRO DE SAN ANTONIO",
"MARIQUITA" : "SAN SEBASTIAN DE MARIQUITA",
"PIENDAMO" : "PIENDAMO - TUNIA",
"SOTARA" : "SOTARA PAISPAMBA",
"SAN JUAN DE RIO SECO" : "SAN JUAN DE RIOSECO",
"CUASPUD" : "CUASPUD CARLOSAMA",
"TOLU VIEJO" : "SAN JOSE DE TOLUVIEJO",
"LOPEZ" : "LOPEZ DE MICAY"}

df["municipio"] = df["municipio"].replace(places_to_change)

In [54]:
df = df.merge(position_df[["municipio" , "LATITUD", "LONGITUD"]], on="municipio", how="left")
df

,arma,departamento,municipio,fecha,genero,grupo_etario,codigo_dane,cantidad,delito,dia,LATITUD,LONGITUD
0,ARMA BLANCA / CORTOPUNZANTE,AMAZONAS,LETICIA,2023-06-13,MASCULINO,ADULTOS,91001000,1,amenza,Martes,-3.530059,-70.045137
1,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,AMAGA,2023-06-26,FEMENINO,ADULTOS,5030000,1,amenza,Lunes,6.032922,-75.708003
2,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,ANGOSTURA,2023-05-05,MASCULINO,ADULTOS,5038000,1,amenza,Viernes,6.861499,-75.358316
3,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,BELLO,2023-02-23,FEMENINO,ADULTOS,5088000,1,amenza,Jueves,6.358841,-75.589736
4,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,CISNEROS,2023-07-25,FEMENINO,ADULTOS,5190000,1,amenza,Martes,6.550065,-75.082541
...,...,...,...,...,...,...,...,...,...,...,...,...
310034,SIN EMPLEO DE ARMAS,VICHADA,PUERTO CARRENO,2023-08-27,FEMENINO,ADULTOS,99001000,1,violencia intrafamiliar,Domingo,5.836530,-68.141222
310035,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALIA,2023-02-07,FEMENINO,ADULTOS,99624000,1,violencia intrafamiliar,Martes,4.968581,-70.659971
310036,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALIA,2023-03-17,FEMENINO,ADULTOS,99624000,1,violencia intrafamiliar,Viernes,4.968581,-70.659971
310037,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALIA,2023-03-31,FEMENINO,ADOLESCENTES,99624000,1,violencia intrafamiliar,Viernes,4.968581,-70.659971


In [55]:
# Add the coordinates to "Purisima" since it isn't on the position df
df.loc[df['municipio'] == 'PURISIMA', 'LATITUD'] = 9.238112
df.loc[df['municipio'] == 'PURISIMA', 'LONGITUD'] = -75.722401

# Importing and cleaning the new DF to know the population per location

In [56]:
population_df = pd.read_excel("../Data/Raw/proyeccion_poblacion.xlsx", skiprows=8, skipfooter=3)
population_df = population_df[(population_df["AÑO"] == 2023) & (population_df["ÁREA GEOGRÁFICA"] == "Total")]
population_df

,DP,DPNOM,MPIO,DPMP,AÑO,ÁREA GEOGRÁFICA,Población
11,5,Antioquia,5001,Medellín,2023,Total,2595300
59,5,Antioquia,5002,Abejorral,2023,Total,21297
107,5,Antioquia,5004,Abriaquí,2023,Total,2841
155,5,Antioquia,5021,Alejandría,2023,Total,4912
203,5,Antioquia,5030,Amagá,2023,Total,32142
...,...,...,...,...,...,...,...
53627,97,Vaupés,97889,Yavaraté (ANM),2023,Total,1200
53675,99,Vichada,99001,Puerto Carreño,2023,Total,22218
53723,99,Vichada,99524,La Primavera,2023,Total,10992
53771,99,Vichada,99624,Santa Rosalía,2023,Total,4555


In [57]:
population_df["DPMP"] = population_df["DPMP"].map(lambda x: str(unidecode(x)).upper())


In [58]:
population_df.to_csv("../Data/Processed/population.csv", index=False)

# Exporting the new df

In [59]:
df.to_csv("../Data/Processed/crimes_enriched.csv", index=False)